In [1]:
import re
import glob
import joblib
import feather
import numpy as np
import pandas as pd
from datetime import date
from currency_converter import CurrencyConverter

from src.clean_data import (
    clean_procedimientos_file,
    convert_to_mxn,
    get_claveuc_proc,
    get_claveuc_nombre,
    get_claveuc_real,
    convert_to_mxn
)

pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 100)

%config IPCompleter.use_jedi = False
%matplotlib inline

In [2]:
file_names = [
    ('../data/raw/Procedimiento de Contratación 2010-2012.xlsx', 2012),
    ('../data/raw/Procedimiento de Contratación 2013.xlsx', 2013),
    ('../data/raw/Procedimiento de Contratación 2014.xlsx', 2014),
    ('../data/raw/Procedimiento de Contratación 2015.xlsx', 2015),
    ('../data/raw/Procedimiento de Contratación 2016.xlsx', 2016),
    ('../data/raw/Contratos 2017.xlsx', 2017),
]

converter_pesos = CurrencyConverter(fallback_on_missing_rate=True, verbose=False, fallback_on_wrong_date=True)

In [2]:
import os

os.path.join('../data/bases/procedimientos/', 'procedimientos_contratacion_2017.xlsx')


'../data/bases/procedimientos/procedimientos_contratacion_2017.xlsx'

## Clean files

In [3]:
df_procedimientos = joblib.Parallel(n_jobs=-1, batch_size=1)(
    joblib.delayed(clean_procedimientos_file)(name, year) for name, year in file_names
)

df_procedimientos = pd.concat(df_procedimientos, axis=0, ignore_index=True)

# Just procedimientos between 2012 and 2017
df_procedimientos = df_procedimientos.loc[(df_procedimientos.FECHA_INICIO.dt.year > 2011), :]

ordered_cols = [
    'GOBIERNO',
    # Dependencia/unidad/responsable
    'DEPENDENCIA',
    'SIGLAS',    
    'NOMBRE_DE_LA_UC',
    'CLAVEUC',
    'RESPONSABLE',
    'ESTRATIFICACION_MUC',
    # proveedor
    'FOLIO_RUPC',
    'PROVEEDOR_CONTRATISTA',
    'ESTATUS_EMPRESA',
    'ESTRATIFICACION_MPC',
    'CLAVE_CARTERA_SHCP',
    # montos
    'IMPORTE_CONTRATO',
    'APORTACION_FEDERAL',
    'MONEDA',
    # datos procedimiento
    'NUMERO_PROCEDIMIENTO',
    'FORMA_PROCEDIMIENTO',
    'TIPO_PROCEDIMIENTO',
    # datos contrato
    'CODIGO_CONTRATO',
    'TITULO_CONTRATO',
    'IDENTIFICADOR_CM',
    'TIPO_CONTRATACION',
    'ESTATUS_CONTRATO',
    'COMPRA_CONSOLIDADA',
    'PLURIANUAL',
    'CARACTER',
    'CONTRATO_MARCO',
    'CONVENIO_MODIFICATORIO',
    # Fechas ordenadas
    'PROC_F_PUBLICACION',
    'FECHA_APERTURA_PROPOSICIONES',
    'EXP_F_FALLO',
    'FECHA_CELEBRACION',
    'FECHA_INICIO',
    'FECHA_FIN',
    # expediente
    'CODIGO_EXPEDIENTE',
    'TITULO_EXPEDIENTE',
    'PLANTILLA_EXPEDIENTE',
    # otros
    'CLAVE_PROGRAMA',
    'CUENTA_ADMINISTRADA_POR',
    'ANUNCIO',
    'ARCHIVADO',
    'SIGLAS_PAIS',
    'RAMO',
    'ORGANISMO',
    'C_EXTERNO',
    'FECHA_ARCHIVO',
]

df_procedimientos = df_procedimientos.loc[:, ordered_cols]
print(df_procedimientos.shape) # 861169

(1015127, 46)


# array(['HOSPITAL GENERAL DE MEXICO "DR EDUARDO LICEAGA"',
#        'HOSPITAL GENERAL DE MEXICO'], dtype=object)

In [4]:
dep_repetidas = {
    'HOSPITAL GENERAL DE MEXICO "DR. EDUARDO LICEAGA"': 'HOSPITAL GENERAL DE MEXICO',
    'INSTITUTO NACIONAL DE REHABILITACION LUIS GUILLERMO IBARRA IBARRA': 'INSTITUTO NACIONAL DE REHABILITACION',
    'CENTRO DE INVESTIGACION EN GEOGRAFIA Y GEOMATICA, "ING. JORGE L. TAMAYO", A.C.': 'CENTRO DE INVESTIGACION EN GEOGRAFIA Y GEOMATICA "ING. JORGE L. TAMAYO", A.C.',
    'TRIBUNAL FEDERAL DE JUSTICIA FISCAL Y ADMINISTRATIVA': 'TRIBUNAL FEDERAL DE JUSTICIA ADMINISTRATIVA',
    'INSTITUTO DE INVESTIGACIONES ELECTRICAS': 'INSTITUTO NACIONAL DE ELECTRICIDAD Y ENERGIAS LIMPIAS'
}

for k, v in dep_repetidas.items():
    df_procedimientos.loc[df_procedimientos.DEPENDENCIA == k, 'DEPENDENCIA'] = v

del dep_repetidas

df_procedimientos = df_procedimientos.assign(
    CLAVEUC_PROC=df_procedimientos.NUMERO_PROCEDIMIENTO.map(get_claveuc_proc)
)
df_procedimientos = df_procedimientos.assign(
    CLAVEUC_NOM=df_procedimientos.NOMBRE_DE_LA_UC.map(get_claveuc_nombre)
)

# Clean CLAVEUC
df_procedimientos.loc[
    df_procedimientos.CLAVEUC.str.len() == 8, 'CLAVEUC'] = df_procedimientos.CLAVEUC.str.pad(9, fillchar='0')
df_procedimientos.loc[
    df_procedimientos.CLAVEUC.str.len() == 7, 'CLAVEUC'] = df_procedimientos.CLAVEUC.str.pad(9, fillchar='0')

unidades_directorio = pd.read_excel(
    '../data/bases/UC_171012060031.xlsx', dtype={'CLAVE_UC': str})
unidades_directorio = set(unidades_directorio.CLAVE_UC.unique())
print(len(unidades_directorio))

print(df_procedimientos.shape)

df_procedimientos = get_claveuc_real(df_procedimientos, unidades_directorio)

# Transform to mexican pesos
monto_en_pesos = convert_to_mxn(
    converter_pesos, df_procedimientos.IMPORTE_CONTRATO.values,
    df_procedimientos.MONEDA.values, df_procedimientos.FECHA_INICIO
)

df_procedimientos = df_procedimientos.assign(IMPORTE_PESOS=monto_en_pesos)

del monto_en_pesos
print(df_procedimientos.shape)

5296
(1015127, 48)
(1015127, 50)


In [5]:
# df_procedimientos.TIPO_PROCEDIMIENTO.value_counts(dropna=False)

## fixes adicionales

In [6]:
# llenar convenio modificatorio 
df_procedimientos = df_procedimientos.assign(
    CONVENIO_MODIFICATORIO=df_procedimientos.CONVENIO_MODIFICATORIO.astype(str)
)
df_procedimientos.loc[df_procedimientos.CONVENIO_MODIFICATORIO == '0.0', 'CONVENIO_MODIFICATORIO'] = 'NO'
df_procedimientos.loc[df_procedimientos.CONVENIO_MODIFICATORIO == '1.0', 'CONVENIO_MODIFICATORIO'] = 'SI'
df_procedimientos.loc[df_procedimientos.CONVENIO_MODIFICATORIO == 'nan', 'CONVENIO_MODIFICATORIO'] = 'NO REPORTADO'

df_procedimientos.loc[
    df_procedimientos.TIPO_PROCEDIMIENTO == 'INVITACION A CUANDO MENOS 3 PERSONAS', 'TIPO_PROCEDIMIENTO'
] = 'INVITACION A CUANDO MENOS TRES'


# # Llenar forma de procedimiento
# df_procedimientos = df_procedimientos.assign(
#     FORMA_PROCEDIMIENTO=df_procedimientos.FORMA_PROCEDIMIENTO.fillna('NO REPORTADA')
# )

# homologar tipo procedimiento
df_procedimientos.loc[
    df_procedimientos.TIPO_PROCEDIMIENTO == 'ADJUDICACION DIRECTA FEDERAL', 'TIPO_PROCEDIMIENTO'
] = 'ADJUDICACION DIRECTA'

codigos_expediente_repeditos = pd.read_csv(
    'codigos_expediente_repetidos.csv', dtype=str, usecols=['CODIGO_EXPEDIENTE']
)
codigos_expediente_repeditos = codigos_expediente_repeditos.CODIGO_EXPEDIENTE.unique()
codigos_expediente_repeditos = set(codigos_expediente_repeditos)
print(df_procedimientos.shape)
df_procedimientos = df_procedimientos.assign(codigos_repetidos=0)
df_procedimientos.loc[
    df_procedimientos.CODIGO_EXPEDIENTE.isin(codigos_expediente_repeditos), 'codigos_repetidos'
] = 1

cols_id = ['CLAVEUC', 'NUMERO_PROCEDIMIENTO', 'CODIGO_EXPEDIENTE', 'CODIGO_CONTRATO', 'codigos_repetidos']
df_procedimientos = df_procedimientos.drop_duplicates(subset=cols_id)
df_procedimientos = df_procedimientos.drop('codigos_repetidos', axis=1)
print(df_procedimientos.shape)

(1015127, 50)
(1014554, 50)


In [9]:
# df_procedimientos.TIPO_PROCEDIMIENTO.value_counts(dropna=False)

In [1]:
1015127 - 1014554

573

In [7]:
df_procedimientos.shape

(1014554, 50)

## Guardar

In [8]:
df_procedimientos = df_procedimientos.drop(['CLAVE_CARTERA_SHCP', 'CLAVEUC_PROC', 'CLAVEUC_NOM'], axis=1)

feather.write_dataframe(
    df_procedimientos, '../data/processed/procedimientos_all_2017_10_29.feather')
df_procedimientos.to_csv(
    '../data/processed/procedimientos_all_2017_10_29.psv', sep='|', encoding='utf-8', index=False, quoting=1)

## Test saved DF

In [9]:
df_test = feather.read_dataframe('../data/processed/procedimientos_all_2017_10_29.feather')
print(df_test.shape)
df_test.head()

(1014554, 47)


,GOBIERNO,DEPENDENCIA,SIGLAS,NOMBRE_DE_LA_UC,CLAVEUC,RESPONSABLE,ESTRATIFICACION_MUC,FOLIO_RUPC,PROVEEDOR_CONTRATISTA,ESTATUS_EMPRESA,ESTRATIFICACION_MPC,IMPORTE_CONTRATO,APORTACION_FEDERAL,MONEDA,NUMERO_PROCEDIMIENTO,FORMA_PROCEDIMIENTO,TIPO_PROCEDIMIENTO,CODIGO_CONTRATO,TITULO_CONTRATO,IDENTIFICADOR_CM,TIPO_CONTRATACION,ESTATUS_CONTRATO,COMPRA_CONSOLIDADA,PLURIANUAL,CARACTER,CONTRATO_MARCO,CONVENIO_MODIFICATORIO,PROC_F_PUBLICACION,FECHA_APERTURA_PROPOSICIONES,EXP_F_FALLO,FECHA_CELEBRACION,FECHA_INICIO,FECHA_FIN,CODIGO_EXPEDIENTE,TITULO_EXPEDIENTE,PLANTILLA_EXPEDIENTE,CLAVE_PROGRAMA,CUENTA_ADMINISTRADA_POR,ANUNCIO,ARCHIVADO,SIGLAS_PAIS,RAMO,ORGANISMO,C_EXTERNO,FECHA_ARCHIVO,CLAVEUC_REAL,IMPORTE_PESOS
0,GE,_GOBIERNO DEL ESTADO DE AGUASCALIENTES,AGS,AGS-INSTITUTO DE EDUCACION DE AGUASCALIENTES-I...,901024986,ANTONIO RUBEN CIENFUEGOS BENAVIDES,NO MIPYME,nan,RAUL EUGENIO ALVAREZ CARRASCO,HABILITADO,MICRO,512930.40,NaN,MXN,IA-901024986-N44-2012,None,INVITACION A CUANDO MENOS TRES,208993,N44 MOB ETC,None,ADQUISICIONES,EXPIRADO,0.0,0.0,None,0.0,NO,2012-10-17 14:51:00,2012-10-23 11:30:00,2012-10-23,NaT,2012-10-23,2012-12-31,271313,ADQUISICION DE MOBILIARIO,Z15122015 04 INVITACION A CUANDO MENOS TRES NA...,None,PoC,None,No,MX,NaN,None,NaN,2012,901024986,512930.40
1,GE,_GOBIERNO DEL ESTADO DE AGUASCALIENTES,AGS,AGS-INSTITUTO DE EDUCACION DE AGUASCALIENTES-I...,901024986,ANTONIO RUBEN CIENFUEGOS BENAVIDES,NO MIPYME,nan,MA DE LA LUZ TISCARENO ELIAS,HABILITADO,NO MIPYME,1887409.40,NaN,MXN,IA-901024986-N44-2012,None,INVITACION A CUANDO MENOS TRES,208992,N44 MOB ETC,None,ADQUISICIONES,EXPIRADO,0.0,0.0,None,0.0,NO,2012-10-17 14:51:00,2012-10-23 11:30:00,2012-10-23,NaT,2012-10-23,2012-10-31,271313,ADQUISICION DE MOBILIARIO,Z15122015 04 INVITACION A CUANDO MENOS TRES NA...,None,PoC,None,No,MX,NaN,None,NaN,2012,901024986,1887409.40
2,GE,_GOBIERNO DEL ESTADO DE AGUASCALIENTES,AGS,AGS-INSTITUTO DE EDUCACION DE AGUASCALIENTES-I...,901024986,ANTONIO RUBEN CIENFUEGOS BENAVIDES,NO MIPYME,nan,PAPELERIA CONSUMIBLES Y ACCESORIOS,HABILITADO,MICRO,1892535.82,NaN,MXN,IA-901024986-N53-2012,None,INVITACION A CUANDO MENOS TRES,257525,IEADJCONT 1140/2012,None,ADQUISICIONES,EXPIRADO,0.0,0.0,None,0.0,NO,2012-11-05 14:34:00,2012-11-12 10:00:00,2012-10-19,NaT,2012-11-12,2012-12-31,268049,IEA PAPELERIA Y CONSUMIBLES,Z15122015 04 INVITACION A CUANDO MENOS TRES NA...,None,PoC,None,No,MX,NaN,None,NaN,2012,901024986,1892535.82
3,GE,_GOBIERNO DEL ESTADO DE AGUASCALIENTES,AGS,AGS-INSTITUTO DE EDUCACION DE AGUASCALIENTES-I...,901024986,ANTONIO RUBEN CIENFUEGOS BENAVIDES,NO MIPYME,19761,TELECOMUNICACIONES MODERNAS,HABILITADO,PEQUENA,24234.00,NaN,MXN,AA-901024986-N31-2012,None,ADJUDICACION DIRECTA,241054,CONSUMIBLES,None,ADQUISICIONES,EXPIRADO,0.0,0.0,None,0.0,NO,2012-09-06 08:49:00,2012-09-07 09:00:00,NaT,NaT,2012-09-11,2012-12-31,252725,COORDINACION ESTATAL DE ASESORIA Y SEGUIMIENTO...,Z15122015 07 ADJUDICACION DIRECTA NACIONAL ART...,None,PoC,None,No,MX,NaN,None,NaN,2012,901024986,24234.00
4,GE,_GOBIERNO DEL ESTADO DE AGUASCALIENTES,AGS,AGS-INSTITUTO DE EDUCACION DE AGUASCALIENTES-I...,901024986,ANTONIO RUBEN CIENFUEGOS BENAVIDES,NO MIPYME,19761,TELECOMUNICACIONES MODERNAS,HABILITADO,PEQUENA,18700.00,NaN,MXN,AA-901024986-N64-2012,None,ADJUDICACION DIRECTA,474294,ADQUISICION DE EQUIPO DE COMPUTO,None,ADQUISICIONES,EXPIRADO,0.0,0.0,None,0.0,NO,2012-12-12 11:38:00,2012-12-14 10:00:00,NaT,NaT,2012-12-17,2012-12-31,320854,EQUIPO DE COMPUTO DESARROLLO EDUCATIVO,Z15122015 07 ADJUDICACION DIRECTA NACIONAL ART...,None,PoC,None,No,MX,NaN,None,NaN,2012,901024986,18700.00


In [11]:
df_procedimientos.TIPO_PROCEDIMIENTO.value_counts(dropna=False)

ADJUDICACION DIRECTA              697038
LICITACION PUBLICA                178498
INVITACION A CUANDO MENOS TRES    135807
OTRO                                2458
LICITACION PUBLICA CON OSD           392
PROYECTO DE CONVOCATORIA             315
NaN                                   41
LICITACION PUBLICA ESTATAL             5
Name: TIPO_PROCEDIMIENTO, dtype: int64

In [1]:
# df_test.loc[df_test.CODIGO_EXPEDIENTE == '193814']

In [2]:
# df_test.loc[df_test.NUMERO_PROCEDIMIENTO == 'AA-012NAW001-N49-2012']

In [3]:
df_test = feather.read_dataframe('../data/bases/procedimientos_all_2017_10_29.feather')
print(df_test.shape)
df_test.head()

(1014554, 47)


,GOBIERNO,DEPENDENCIA,SIGLAS,NOMBRE_DE_LA_UC,CLAVEUC,RESPONSABLE,ESTRATIFICACION_MUC,FOLIO_RUPC,PROVEEDOR_CONTRATISTA,ESTATUS_EMPRESA,ESTRATIFICACION_MPC,IMPORTE_CONTRATO,APORTACION_FEDERAL,MONEDA,NUMERO_PROCEDIMIENTO,FORMA_PROCEDIMIENTO,TIPO_PROCEDIMIENTO,CODIGO_CONTRATO,TITULO_CONTRATO,IDENTIFICADOR_CM,TIPO_CONTRATACION,ESTATUS_CONTRATO,COMPRA_CONSOLIDADA,PLURIANUAL,CARACTER,CONTRATO_MARCO,CONVENIO_MODIFICATORIO,PROC_F_PUBLICACION,FECHA_APERTURA_PROPOSICIONES,EXP_F_FALLO,FECHA_CELEBRACION,FECHA_INICIO,FECHA_FIN,CODIGO_EXPEDIENTE,TITULO_EXPEDIENTE,PLANTILLA_EXPEDIENTE,CLAVE_PROGRAMA,CUENTA_ADMINISTRADA_POR,ANUNCIO,ARCHIVADO,SIGLAS_PAIS,RAMO,ORGANISMO,C_EXTERNO,FECHA_ARCHIVO,CLAVEUC_REAL,IMPORTE_PESOS
0,GE,_GOBIERNO DEL ESTADO DE AGUASCALIENTES,AGS,AGS-INSTITUTO DE EDUCACION DE AGUASCALIENTES-I...,901024986,ANTONIO RUBEN CIENFUEGOS BENAVIDES,NO MIPYME,nan,RAUL EUGENIO ALVAREZ CARRASCO,HABILITADO,MICRO,512930.40,NaN,MXN,IA-901024986-N44-2012,None,INVITACION A CUANDO MENOS TRES,208993,N44 MOB ETC,None,ADQUISICIONES,EXPIRADO,0.0,0.0,None,0.0,NO,2012-10-17 14:51:00,2012-10-23 11:30:00,2012-10-23,NaT,2012-10-23,2012-12-31,271313,ADQUISICION DE MOBILIARIO,Z15122015 04 INVITACION A CUANDO MENOS TRES NA...,None,PoC,None,No,MX,NaN,None,NaN,2012,901024986,512930.40
1,GE,_GOBIERNO DEL ESTADO DE AGUASCALIENTES,AGS,AGS-INSTITUTO DE EDUCACION DE AGUASCALIENTES-I...,901024986,ANTONIO RUBEN CIENFUEGOS BENAVIDES,NO MIPYME,nan,MA DE LA LUZ TISCARENO ELIAS,HABILITADO,NO MIPYME,1887409.40,NaN,MXN,IA-901024986-N44-2012,None,INVITACION A CUANDO MENOS TRES,208992,N44 MOB ETC,None,ADQUISICIONES,EXPIRADO,0.0,0.0,None,0.0,NO,2012-10-17 14:51:00,2012-10-23 11:30:00,2012-10-23,NaT,2012-10-23,2012-10-31,271313,ADQUISICION DE MOBILIARIO,Z15122015 04 INVITACION A CUANDO MENOS TRES NA...,None,PoC,None,No,MX,NaN,None,NaN,2012,901024986,1887409.40
2,GE,_GOBIERNO DEL ESTADO DE AGUASCALIENTES,AGS,AGS-INSTITUTO DE EDUCACION DE AGUASCALIENTES-I...,901024986,ANTONIO RUBEN CIENFUEGOS BENAVIDES,NO MIPYME,nan,PAPELERIA CONSUMIBLES Y ACCESORIOS,HABILITADO,MICRO,1892535.82,NaN,MXN,IA-901024986-N53-2012,None,INVITACION A CUANDO MENOS TRES,257525,IEADJCONT 1140/2012,None,ADQUISICIONES,EXPIRADO,0.0,0.0,None,0.0,NO,2012-11-05 14:34:00,2012-11-12 10:00:00,2012-10-19,NaT,2012-11-12,2012-12-31,268049,IEA PAPELERIA Y CONSUMIBLES,Z15122015 04 INVITACION A CUANDO MENOS TRES NA...,None,PoC,None,No,MX,NaN,None,NaN,2012,901024986,1892535.82
3,GE,_GOBIERNO DEL ESTADO DE AGUASCALIENTES,AGS,AGS-INSTITUTO DE EDUCACION DE AGUASCALIENTES-I...,901024986,ANTONIO RUBEN CIENFUEGOS BENAVIDES,NO MIPYME,19761,TELECOMUNICACIONES MODERNAS,HABILITADO,PEQUENA,24234.00,NaN,MXN,AA-901024986-N31-2012,None,ADJUDICACION DIRECTA,241054,CONSUMIBLES,None,ADQUISICIONES,EXPIRADO,0.0,0.0,None,0.0,NO,2012-09-06 08:49:00,2012-09-07 09:00:00,NaT,NaT,2012-09-11,2012-12-31,252725,COORDINACION ESTATAL DE ASESORIA Y SEGUIMIENTO...,Z15122015 07 ADJUDICACION DIRECTA NACIONAL ART...,None,PoC,None,No,MX,NaN,None,NaN,2012,901024986,24234.00
4,GE,_GOBIERNO DEL ESTADO DE AGUASCALIENTES,AGS,AGS-INSTITUTO DE EDUCACION DE AGUASCALIENTES-I...,901024986,ANTONIO RUBEN CIENFUEGOS BENAVIDES,NO MIPYME,19761,TELECOMUNICACIONES MODERNAS,HABILITADO,PEQUENA,18700.00,NaN,MXN,AA-901024986-N64-2012,None,ADJUDICACION DIRECTA,474294,ADQUISICION DE EQUIPO DE COMPUTO,None,ADQUISICIONES,EXPIRADO,0.0,0.0,None,0.0,NO,2012-12-12 11:38:00,2012-12-14 10:00:00,NaT,NaT,2012-12-17,2012-12-31,320854,EQUIPO DE COMPUTO DESARROLLO EDUCATIVO,Z15122015 07 ADJUDICACION DIRECTA NACIONAL ART...,None,PoC,None,No,MX,NaN,None,NaN,2012,901024986,18700.00


In [7]:
df_test.loc[df_test.NUMERO_PROCEDIMIENTO == 'LO-019GYR052-N1-2013']

,GOBIERNO,DEPENDENCIA,SIGLAS,NOMBRE_DE_LA_UC,CLAVEUC,RESPONSABLE,ESTRATIFICACION_MUC,FOLIO_RUPC,PROVEEDOR_CONTRATISTA,ESTATUS_EMPRESA,ESTRATIFICACION_MPC,IMPORTE_CONTRATO,APORTACION_FEDERAL,MONEDA,NUMERO_PROCEDIMIENTO,FORMA_PROCEDIMIENTO,TIPO_PROCEDIMIENTO,CODIGO_CONTRATO,TITULO_CONTRATO,IDENTIFICADOR_CM,TIPO_CONTRATACION,ESTATUS_CONTRATO,COMPRA_CONSOLIDADA,PLURIANUAL,CARACTER,CONTRATO_MARCO,CONVENIO_MODIFICATORIO,PROC_F_PUBLICACION,FECHA_APERTURA_PROPOSICIONES,EXP_F_FALLO,FECHA_CELEBRACION,FECHA_INICIO,FECHA_FIN,CODIGO_EXPEDIENTE,TITULO_EXPEDIENTE,PLANTILLA_EXPEDIENTE,CLAVE_PROGRAMA,CUENTA_ADMINISTRADA_POR,ANUNCIO,ARCHIVADO,SIGLAS_PAIS,RAMO,ORGANISMO,C_EXTERNO,FECHA_ARCHIVO,CLAVEUC_REAL,IMPORTE_PESOS
185523,APF,INSTITUTO MEXICANO DEL SEGURO SOCIAL,IMSS,IMSS-00641230,019GYR052,JOSE ANGEL PATINO NAVARRO,PEQUENA,nan,PEDRO CESAR INFANTE CONTRERAS,HABILITADO,PEQUENA,4720050.48,6320741.0,MXN,LO-019GYR052-N1-2013,ELECTRONICA,LICITACION PUBLICA,297965,MANTENIMIENTO A OBRA CIVIL E INSTALACIONES ZON...,None,OBRA PUBLICA,EXPIRADO,0.0,0.0,NACIONAL,0.0,NO REPORTADO,2013-01-15 02:00:00,2013-01-31 10:00:00,2013-02-08,2013-02-20,2013-02-20,2013-10-09,327720,MANTENIMIENTO A ZONA I MORELIA IMSS MICH,16 LICITACION PUBLICA NACIONAL LOPSRM,None,PoC,None,No,None,12.0,None,NaN,2013,019GYR052,4720050.48
185524,APF,INSTITUTO MEXICANO DEL SEGURO SOCIAL,IMSS,IMSS-00641230,019GYR052,JOSE ANGEL PATINO NAVARRO,PEQUENA,nan,MANRIQUEZ ARQUITECTOS,HABILITADO,MICRO,2345570.49,6320741.0,MXN,LO-019GYR052-N1-2013,ELECTRONICA,LICITACION PUBLICA,297964,MANTENIMIENTO A OBRA CIVIL E INSTALACIONES HG...,None,OBRA PUBLICA,EXPIRADO,0.0,0.0,NACIONAL,0.0,NO REPORTADO,2013-01-15 02:00:00,2013-01-31 10:00:00,2013-02-08,2013-02-20,2013-02-20,2013-10-09,327720,MANTENIMIENTO A ZONA I MORELIA IMSS MICH,16 LICITACION PUBLICA NACIONAL LOPSRM,None,PoC,None,No,None,12.0,None,NaN,2013,019GYR052,2345570.49


In [3]:
df_test.TIPO_PROCEDIMIENTO.value_counts(dropna=False)

ADJUDICACION DIRECTA                    697038
LICITACION PUBLICA                      178498
INVITACION A CUANDO MENOS 3 PERSONAS    135807
OTRO                                      2458
LICITACION PUBLICA CON OSD                 392
PROYECTO DE CONVOCATORIA                   315
NaN                                         41
LICITACION PUBLICA ESTATAL                   5
Name: TIPO_PROCEDIMIENTO, dtype: int64

In [4]:
df_test.TIPO_CONTRATACION.value_counts(dropna=False)

ADQUISICIONES                       500491
SERVICIOS                           352886
OBRA PUBLICA                        129409
SERVICIOS RELACIONADOS CON LA OP     25639
ARRENDAMIENTOS                        6129
Name: TIPO_CONTRATACION, dtype: int64

In [ ]:
# Read valid unidades compradoras
# unidades_directorio = pd.read_excel(
#     '../data/raw/Directorio Unidades Compradoras Gobierno Federal.xls', dtype=str)
# unidades_directorio = set(
#     unidades_directorio.loc[:, 'Clave UC en CompraNet'].unique())

# unidades_est = pd.read_excel(
#     '../data/raw/Directorio Unidades Compradoras Gobierno Estatal.xls', dtype=str)
# unidades_est = set(
#     unidades_est.loc[:, 'Clave UC en CompraNet'].unique())

# unidades_mun = pd.read_excel(
#     '../data/raw/Directorio Unidades Compradoras Gobierno Municipal.xls', dtype=str)
# unidades_mun = set(
#     unidades_mun.loc[:, 'Clave UC en CompraNet'].unique())

# unidades_directorio = unidades_directorio | unidades_est | unidades_mun